Copyright (c) 2020, NVIDIA CORPORATION.
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [3]:
import os, time
os.environ["CUDA_VISIBLE_DEVICES"]="0"
start = time.time()
very_start = time.time()

In [4]:
#import pandas as pd, 
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.max_rows', 500)
import cudf, cupy, time, rmm
cudf.__version__

'0.16.0'

In [5]:
import dask as dask, dask_cudf
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
import subprocess

In [6]:
'''cluster = LocalCUDACluster(ip='10.2.61.36')#,protocol="ucx", 
                           #rmm_pool_size="26GB",
                           #enable_tcp_over_ucx=True, enable_nvlink=True)
#cluster = LocalCUDACluster()
client = Client(cluster)
client'''

'cluster = LocalCUDACluster(ip=\'10.2.61.36\')#,protocol="ucx", \n                           #rmm_pool_size="26GB",\n                           #enable_tcp_over_ucx=True, enable_nvlink=True)\n#cluster = LocalCUDACluster()\nclient = Client(cluster)\nclient'

In [7]:
%%time
path = './dask_input/step1_output'
df = dask_cudf.read_parquet(f'{path}/*.parquet')
df, = dask.persist(df)
#_ = wait(df)
df.head()

CPU times: user 2.79 s, sys: 920 ms, total: 3.71 s
Wall time: 3.72 s


,hashtags,links,domains,timestamp,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_follower_count,b_following_count,...,retweet,retweet_comment,like,id,language,tweet_type,media,tweet_id,a_user_id,b_user_id
0,5F5E25FDD0B767C4BB6DE6473DC6AE33,<NA>,<NA>,1582018502,3388192,0,True,1312003421,4,206,...,0,0,0,13801804,54,2,8,1302828,553866,228624
1,E6D1192E7880DEE2E18C3D4489970918,<NA>,<NA>,1581798652,916,891,False,1496942442,504,938,...,0,0,0,8752519,54,2,4,1056460,430744,356802
2,<NA>,D1749EC58B796AA39D94DBE4153C1F62,7471C2E3375040BA50F8044E5D52B083,1581764307,171597,423,True,1462544521,76,69,...,0,0,0,8180770,54,2,12,1495267,645271,273905
3,3E73FEBD954B9E5A5C85F2AFFD5ED49D\tA62335257DDD...,<NA>,<NA>,1581869312,3190,24,False,1510154461,632,18,...,0,0,0,3728019,31,1,12,1321218,452570,550597
4,<NA>,<NA>,<NA>,1581904827,1760209,1,False,1416406065,255,113,...,0,0,0,11920974,47,2,4,986636,647160,122918


In [8]:
%%time
def count_token(ds,token):
    not_null = ds.isnull()==0
    return ((ds.str.count(token)+1)*not_null).fillna(0)
df['len_hashtags'] = df['hashtags'].map_partitions(lambda ds: count_token(ds,'\t'))
df['len_domains']  = df['domains'].map_partitions(lambda ds: count_token(ds,'\t'))
df['len_links']    = df['links'].map_partitions(lambda ds: count_token(ds,'\t'))

df, = dask.persist(df)
#_ = wait(df)
df.head()

CPU times: user 332 ms, sys: 84 ms, total: 416 ms
Wall time: 414 ms


,hashtags,links,domains,timestamp,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_follower_count,b_following_count,...,id,language,tweet_type,media,tweet_id,a_user_id,b_user_id,len_hashtags,len_domains,len_links
0,5F5E25FDD0B767C4BB6DE6473DC6AE33,<NA>,<NA>,1582018502,3388192,0,True,1312003421,4,206,...,13801804,54,2,8,1302828,553866,228624,1,0,0
1,E6D1192E7880DEE2E18C3D4489970918,<NA>,<NA>,1581798652,916,891,False,1496942442,504,938,...,8752519,54,2,4,1056460,430744,356802,1,0,0
2,<NA>,D1749EC58B796AA39D94DBE4153C1F62,7471C2E3375040BA50F8044E5D52B083,1581764307,171597,423,True,1462544521,76,69,...,8180770,54,2,12,1495267,645271,273905,0,1,1
3,3E73FEBD954B9E5A5C85F2AFFD5ED49D\tA62335257DDD...,<NA>,<NA>,1581869312,3190,24,False,1510154461,632,18,...,3728019,31,1,12,1321218,452570,550597,2,0,0
4,<NA>,<NA>,<NA>,1581904827,1760209,1,False,1416406065,255,113,...,11920974,47,2,4,986636,647160,122918,0,0,0


In [9]:
def most_frequent_token(df, col, idcol, outcol, sep='\t'):
    ds = df[[idcol,col]].compute()
    tokens = ds[col].str.tokenize(sep)
    
    token_counts = ds[col].str.token_count('\t').to_frame()
    token_counts[idcol] = ds[idcol]
    del ds
    token_rows = reconstruct_token_rows(token_counts,col,idcol)
    del token_counts
    
    tokens = tokens.to_frame()
    tokens.columns = ['token']
    tokens['token'],_ = tokens['token'].factorize()
    tokens['token'] = tokens['token']+1
    tokens[idcol] = token_rows
    
    global_token_counts = tokens['token'].value_counts().reset_index()
    global_token_counts.columns = ['token','count']
    
    tokens = tokens.merge(global_token_counts,on='token',how='left')
    del global_token_counts
    
    token_counts = tokens.groupby(idcol).agg({'count':'max'}).reset_index()
    tokens = tokens.merge(token_counts,on=idcol,how='left')
    del token_counts
    mask = tokens.count_x == tokens.count_y
    tokens = tokens[mask]
    tokens = tokens.drop(['count_x','count_y'],axis=1)
    tokens = tokens.drop_duplicates(subset=['token',idcol])
    tokens = tokens.rename(columns={'token':outcol})
    
    if outcol == col:
        df = df.drop(col,axis=1)
    df = df.merge(tokens,on=idcol,how='left')
    df[outcol] = df[outcol].fillna(0)
    del tokens
    return df

def reconstruct_token_rows(token_counts,col,idcol):
    ids = cupy.asnumpy(token_counts[idcol].values)
    rep = cupy.asnumpy(token_counts[col].values)
    token_rows = np.repeat(ids,rep)
    token_rows = cupy.asarray(token_rows)
    return token_rows


In [10]:
%%time
for col in ['domains','links','hashtags']:
    df = most_frequent_token(df, col=col, idcol='id', outcol=col, sep='\t')
    df, = dask.persist(df)
    #_ = wait(df)
df.head()

CPU times: user 3.18 s, sys: 1.92 s, total: 5.09 s
Wall time: 5.1 s


,timestamp,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_follower_count,b_following_count,b_is_verified,b_account_creation,b_follows_a,...,media,tweet_id,a_user_id,b_user_id,len_hashtags,len_domains,len_links,domains,links,hashtags
0,1581561262,413541,41702,True,1350796987,631,639,False,1341899529,False,...,12,3930251,645511,260794,2,0,0,0,0,141479
1,1581605897,798363,3,True,1507525603,68,296,False,1439433727,False,...,6,3933361,477469,274216,3,0,0,0,0,970046
2,1581854175,1969,1523,False,1350039207,2742,1550,False,1270234613,True,...,6,4411215,625348,667622,1,0,0,0,0,148692
3,1582026603,337,12,False,1340883424,685,1102,False,1405816969,False,...,4,4863306,496434,86549,0,0,0,0,0,0
4,1581930519,194314,2140,False,1436790503,508,874,False,1348926123,False,...,12,3993866,415686,546378,0,0,0,0,0,0


In [11]:
%%time

TRAIN_SIZE = 121386431 # TODO correct?
VALID_SIZE = 12434735

train = df[df['id']<=TRAIN_SIZE] # df['id'].min() = 1
test0 = df[(df['id']>TRAIN_SIZE)&(df['id']<=TRAIN_SIZE+VALID_SIZE)]
test1 = df[df['id']>TRAIN_SIZE+VALID_SIZE]

print(len(train),len(test0),len(test1))

21043825 0 0
CPU times: user 80 ms, sys: 104 ms, total: 184 ms
Wall time: 155 ms


In [12]:
%%time
path = './dask_input/step2_output'

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.91 µs


In [13]:
%%time
train.to_parquet(f'{path}/train.parquet', write_index=False)
test0.to_parquet(f'{path}/valid.parquet', write_index=False)
test1.to_parquet(f'{path}/test.parquet', write_index=False)

CPU times: user 980 ms, sys: 1.87 s, total: 2.85 s
Wall time: 1.16 s


In [14]:
print('This notebook took %.1f minutes'%((time.time()-very_start)/60.))

This notebook took 0.2 minutes
